In [1]:
#importing the required libraries

import sys
import os
import json,gzip
from pprint import pprint
from elasticsearch import Elasticsearch,helpers


In [2]:
# Elastic search configuation

es = Elasticsearch("http://localhost:9200")

# checking whether the connection is established

es.ping()

True

In [4]:
#opening the file and store it in the data variable.

data = open(r"C:\Users\renji\Downloads\iris data set\sample-1M.jsonl")

#using a for loop to iterate over each line 

for each_line in data:
    each_line = each_line.replace("\n", "")
    doc = {"data": json.loads(each_line)}
    
#indexing the document using Elastic search index function

    es.index(index='articles1', doc_type='_doc', body=doc)  
    
print('finished upload')

finished upload


# Elasticsearch Code

pyhton is used till indexing ,  Dev Tool is used to do rest tasks.
Codes corresponding to each task is shown below. It will work in the Dev Tool Console.

In [ ]:
    
#issue: Elasticsearch can't update non dynamic settings: close the index and update again
#closing an index
POST /articles/_close

#Tokenization and Case folding 

PUT articles/_settings
{
  "analysis" : {
    "analyzer":{
      "my_analyzer":{
        "tokenizer":"whitespace",
           "filter": ["lowercase","asciifolding","word_delimiter"
            ]
      }
    }
  }
}

#test string

POST articles/_analyze
{
  "analyzer": "my_analyzer",
  "text": "The IÀNS India-Private+ Limited"
}



In [ ]:
#Selecting Keywords 

#stop word removal using "stop token filter"


PUT articles/_settings
{
  "analysis" : {
    "analyzer":{
      "my_analyzer":{
        "tokenizer":"whitespace",
           "filter": ["lowercase","asciifolding","word_delimiter","stop"
            ]
      }
    }
  }
}

#test string

POST articles/_analyze
{
  "analyzer": "my_analyzer",
  "text": "The IÀNS India-Private+ Limited"
}



In [ ]:
#N-gram Tokenizer:

#creating custom tokenizer for N gram called "my_tokenizer"

PUT articles/_settings
{
  "settings": {
    "analysis": {
      "analyzer": {
        "my_analyzer": {
          "tokenizer": "my_tokenizer",
           "filter": ["lowercase","asciifolding","word_delimiter","stop"
            ]
        }
      },
      "tokenizer": {
        "my_tokenizer": {
          "type": "ngram",
          "min_gram": 3,
          "max_gram": 3,
          "token_chars": [
            "letter",
            "digit"
          ]
        }
      }
    }
  }
}

#test string

POST articles/_analyze
{
  "analyzer": "my_analyzer",
  "text": "The IÀNS India-Private+ Limited"
}


In [ ]:
#TF-IDF Score  using similarity module:

PUT /articles/_settings
{
      "similarity": {
          "default" : {
            "type" : "BM25",
            "b": 0.75,
            "k1": 1.2
          }
        }
      }
    }
  }
}

In [ ]:
#stemmimg :

PUT /articles/_settings
{
  "settings": {
    "analysis": {
      "analyzer": {
        "my_analyzer2": {
          "tokenizer": "whitespace",
          "filter": [
            "lowercase",
            "kstem"
          ]
        }
      }
    }
  }
}

## Searching

In [ ]:


#1.	Find the articles published  in  a particular date:

GET articles/_search
{
    "query": {
      
        "range" : {
            "data.published": {
                "gte" : "2015-09-29"
            }
        }
    }
}

In [ ]:
#2.Find an article published in a specific date and source.

POST articles/_search
{
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "data.source": "yahoo sports"
          }
        },
        {
          "range": {
            "data.published": {
              "gte": "2015-09-01"
            }
          }
        }
      ]
    }
  }
}

In [ ]:
#3.Find the articles related to a particular content and published in a specific date and source.

POST articles/_search
{
    "query": {
        "bool": {
            "must": [
              {
                "bool": {
                    "should": [{
                        "match": {
                            "data.title": "Randy Moss  boot camp "
                        }
                    }
                    ]
                }
            }, 
            {
                "bool": {
                    "should": [
                      {
                        "match": {
                            "data.source": "yahoo sports"
                        }
                    }
                    ]
                }
            }, {
                "bool": {
                    "must": [
                      {
                        "range": {
                          "data.published": {
                            "gte": "2015-09-28"
                          }
                        }
                      }
                      ]
                }
            }]
        }
    }
}
